In [22]:
import numpy as np
import pandas as pd
import csv
from scipy.stats import kruskal,mannwhitneyu

In [2]:
def soft_max (x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [6]:
datacsv = open("../General Survey.csv")
reader = list(csv.reader(datacsv, delimiter=',', quotechar='"'))
fields = [x.strip() for x in reader.pop(0)]
print(fields)

['Timestamp', 'Please rank the following (No repeat ranks, must rank each row) [Production Budget]', 'Please rank the following (No repeat ranks, must rank each row) [Producer]', 'Please rank the following (No repeat ranks, must rank each row) [Average Critic Score (Rotten Tomatoes)]', 'Please rank the following (No repeat ranks, must rank each row) [IMDB Rating]', 'Please rank the following (No repeat ranks, must rank each row) [IMDB Votes]', 'Please rank the following (No repeat ranks, must rank each row) [Average Audience Score (Rotten Tomatoes)]', 'Please rank the following (No repeat ranks, must rank each row) [Return on Investment]', 'Please rank the following (No repeat ranks, must rank each row) [Worldwide Gross]', 'How much do you enjoy researching movies, such as browsing IMDB, Wikipedia pages, and looking up actors?', 'How much do you enjoy watching movies?', 'How often do you watch the Oscars or similar movie award shows?', 'What is your current age?', 'Are you currently at

In [7]:
reader = [[cell.split("(")[0].strip() for cell in col]for col in reader]

In [47]:
ratings = pd.DataFrame(reader).values[:,1:-5].astype(int)
ratings = ratings.T
ratings = np.delete(ratings,1,axis=0) # removing producer
trimmed_fields = [x.split("[")[1].split("]")[0].strip().split('(')[0] for x in fields[1:-5]]
trimmed_fields.pop(1) # removing producer
print(trimmed_fields)
means = [x.mean() for x in ratings]
# print(means)

softs = soft_max(means)
print("Soft Max Values")
zipped = [[f,v]for f,v in zip(trimmed_fields,softs)]
zipped.sort(key=lambda x: x[0], reverse=True)
for f,v in zipped:
    print("%s: %s" % (f,v))

['Production Budget', 'Average Critic Score ', 'IMDB Rating', 'IMDB Votes', 'Average Audience Score ', 'Return on Investment', 'Worldwide Gross']
Soft Max Values
Worldwide Gross: 0.167137582418
Return on Investment: 0.0607032238123
Production Budget: 0.0336580824017
IMDB Votes: 0.0209448803779
IMDB Rating: 0.101374068127
Average Critic Score : 0.557769349727
Average Audience Score : 0.0584128131355


In [49]:
logregcoefs = [['IMDB Rating', -0.34300168957186966], ['IMDB Votes', 1.3669196119522085], ['Average Critic Score', 0.57744046973782626], ['Average Audience Score', -0.40653727105115345], ['Production Budget', -0.27290349924072632], ['Worldwide Gross', 0.69783373108027336], ['Return on Investment', 1.3457350571892375]]
gamcoefs = [['IMDB Votes', -0.726622204], ['IMDB Rating',0.157007000], ['Average Critic Score',0.109126851], ['Average Audience Score',0.003958049], ['Production Budget',-7.045378456], ['Worldwide Gross',5.623981533], ['Return on Investment',0.130592474]]
gamcoefs = np.array(gamcoefs)
logregcoefs = np.array(logregcoefs)

mergedregcoefs = np.array([[x[0], (float(x[1])+float(y[1]))/2] for x,y in zip(gamcoefs,logregcoefs)])
mergedregcoefs[:,1] = soft_max(mergedregcoefs[:,1].astype(float))

mergedregcoefs = mergedregcoefs.tolist()
mergedregcoefs.sort(key=lambda x: x[0], reverse=True)
for f,v in mergedregcoefs:
    print("%s: %s" % (f,v))
    

# logregcoefs[:,1] = soft_max(logregcoefs[:,1].astype(float))
# gamcoefs[:,1] = soft_max(gamcoefs[:,1].astype(float))

# logregcoefs = logregcoefs.tolist()
# logregcoefs.sort(key=lambda x: x[0], reverse=True)
# for f,v in logregcoefs:
#     print("%s: %s" % (f,v))

Worldwide Gross: 0.7693374461761608
Return on Investment: 0.06822327746934165
Production Budget: 0.0008398609599352032
IMDB Votes: 0.019102325968666383
IMDB Rating: 0.0698664357636996
Average Critic Score: 0.04596617101656862
Average Audience Score: 0.026664482645627638


In [48]:
gamcoefs = gamcoefs.tolist()
gamcoefs.sort(key=lambda x: x[0], reverse=True)
for f,v in gamcoefs:
    print("%s: %s" % (f,v))

Worldwide Gross: 5.623981533
Return on Investment: 0.130592474
Production Budget: -7.045378456
IMDB Votes: -0.726622204
IMDB Rating: 0.157007
Average Critic Score: 0.109126851
Average Audience Score: 0.003958049


In [51]:
zipped1 = [z[1] for z in zipped] # get just the values
mergedreg = [float(x[1]) for x in mergedregcoefs]
logreg1 = [float(l[1]) for l in logregcoefs]
gam1 = [float(g[1]) for g in gamcoefs]

test = kruskal(np.array(zipped1),np.array(mergedreg))
# test = kruskal(np.array(zipped1),np.array(logreg1), np.array(gam1))
# test2 = mannwhitneyu(np.array(zipped1),np.array(logreg1))
print(test)


KruskalResult(statistic=0.689795918367345, pvalue=0.40623380511079543)
